# Import Library

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import random
from collections import OrderedDict

# Data Preprocessing

In [3]:
path = './digit-recognizer/'
test_data = pd.read_csv(path+'test.csv')
train_data = pd.read_csv(path+'train.csv')
# Ground Truth of training data
groundtruth = np.array(train_data['label'])
train_data = train_data.drop(['label'], axis=1)

In [4]:
# Transfer dataframe to np array
train_set = np.array(train_data)
test_set = np.array(test_data)
# Reshape of np array to matrix form [1*28*28] for each black-white image
train_set = np.reshape(train_set, [train_set.shape[0], 1, 28, 28])
test_set = np.reshape(test_set, [test_set.shape[0], 1, 28, 28])
print(train_set.shape)
print(test_set.shape)
# Data Normalization
nor_train = np.linalg.norm(train_set)
train_set = train_set / nor_train
nor_test = np.linalg.norm(test_set)
test_set = test_set / nor_test
# Split training data into train_set and val_set 4:1 using 5-Fold Cross Validation
k = 5
epoch = 100
batch_size = 64
splits = KFold(n_splits=k, shuffle=True, random_state=35)

(42000, 1, 28, 28)
(28000, 1, 28, 28)


In [5]:
# Building my own dataset
class digitDataset(Dataset):
    def __init__(self, x, y=None):
        self.data = torch.from_numpy(x).float()
        if y is not None:
            y = y.astype(int)
            self.label = torch.LongTensor(y)
        else: self.label = None
    def __getitem__(self, index):
        if self.label is not None:
            return self.data[index], self.label[index]
        else: return self.data[index]
    def __len__(self):
        return len(self.data)

# CNN Model Architecture

In [6]:
# Loss Function
criterion = nn.CrossEntropyLoss()
# Checking using device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Dictionary for saving each fold's train and val loss
foldperf = {}

In [15]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.CNN_layer = nn.Sequential(
            nn.Conv2d(in_channels= 1, out_channels= 64, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 64, out_channels= 64, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(in_channels= 64, out_channels= 96, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(96),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 96, out_channels= 64, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 64, out_channels= 96, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(96),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(in_channels= 96, out_channels= 128, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 128, out_channels= 96, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(96),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 96, out_channels= 128, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(in_channels= 128, out_channels= 256, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 256, out_channels= 192, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(192),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 192, out_channels= 256, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(in_channels= 256, out_channels= 448, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(448),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 448, out_channels= 256, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels= 256, out_channels= 448, kernel_size= 3, stride= 1, padding= 1),
            nn.BatchNorm2d(448),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )
        self.FC_layer = nn.Sequential(
            nn.Linear(448, 10),
            nn.Softmax()
        )
    def forward(self, x):
        x = self.CNN_layer(x)
        x = self.FC_layer(x)
        return x

In [ ]:
from torchsummary import summary

test_model = CNNModel()
summary(test_model, (1, 28, 28), device="cpu")

# K Fold Cross Validation

In [49]:
def train_epoch(model, device, dataloader, loss_fn, optimizer):
    train_loss, train_correct = 0.0, 0
    model.train()
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(output.data, 1)
        train_correct += (predictions == labels).sum().item()
    return train_loss, train_correct

In [50]:
def valid_epoch(model, device, dataloader, loss_fn):
    valid_loss, val_correct = 0.0, 0
    model.eval()
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        loss = loss_fn(output, labels)
        valid_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(output.data, 1)
        val_correct += (predictions == labels).sum().item()
    return valid_loss, val_correct

In [51]:
for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(train_set.shape[0]))):
    print('Fold {}'.format(fold + 1))
    train_dset = digitDataset(train_set, groundtruth)
    train_loader = DataLoader(train_dset, batch_size=batch_size, sampler=train_idx)
    val_loader = DataLoader(train_dset, batch_size=batch_size, sampler=val_idx)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model =CNNModel()
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-5)

    history = {'train_loss': [], 'val_loss': [],'train_acc':[],'val_acc':[]}

    for epoch in range(epoch):
        train_loss, train_correct = train_epoch(model, device, train_loader, criterion, optimizer)
        val_loss, val_correct = valid_epoch(model, device, val_loader, criterion)

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        val_loss = val_loss / len(val_loader.sampler)
        val_acc = val_correct / len(val_loader.sampler) * 100

        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Validae Loss:{:.3f} AVG Training Acc {:.2f} % AVG Validate Acc {:.2f} %".format(epoch + 1,
                                                                                                             epoch,
                                                                                                             train_loss,
                                                                                                             val_loss,
                                                                                                             train_acc,
                                                                                                             val_acc))
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)

    foldperf['fold{}'.format(fold+1)] = history  

torch.save(model,'k_cross_CNN.pt')

Fold 1


TypeError: 'tuple' object is not callable